In [ ]:
import os
import base64
import struct

import bson.binary
from pymongo.mongo_client import MongoClient
from bson.binary import Binary

from dotenv import load_dotenv

load_dotenv()


In [ ]:
uri = os.environ['DB_URI']

# Create a new client and connect to the server
client = MongoClient(uri)

In [ ]:
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    # Verbindung zur MongoDB herstellen
    db = client['rosen']
    sammlung = db['big_data']

    # Alle Dokumente in der Sammlung durchgehen
    for dokument in sammlung.find():
        if "measuring_points" in dokument:
            # Iteriere durch jeden Eintrag in der Liste
            for entry in dokument["measuring_points"]:
                # Überprüfe, ob das Feld "magnetization" im Eintrag vorhanden ist
                if "magnetization" in entry:
                    # Base64-Daten aus dem Eintrag extrahieren
                    base64_data = entry["magnetization"]

                    if base64_data and not isinstance(base64_data, float):
                        try:

                            # Base64 zu Binär konvertieren
                            bin_data = float(base64_data)

                            # Das Dokument aktualisieren und die BinData durch Float ersetzen
                            sammlung.update_one(
                                {"_id": dokument["_id"], "measuring_points.magnetization": base64_data},
                                {"$set": {"measuring_points.$.magnetization": bin_data}})
                        except:
                            print(f"Fehler beim Dekodieren der Base64-Daten in Dokument mit ID {dokument['_id']}")

                            print(f"Aktualisiere Dokument mit ID {dokument['_id']}")
